In [1]:
import pandas as pd
import re
%matplotlib inline
%env JOBLIB_TEMP_FOLDER=/tmp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from tqdm import tqdm


import nltk
from nltk.corpus import stopwords
from os.path import join
from glob import glob

from time import time
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN, AffinityPropagation, SpectralClustering, KMeans

env: JOBLIB_TEMP_FOLDER=/tmp


In [5]:
def preprocessing_texts(df):
    df = df['text']
    df = re.sub(r'\(http[\S]*\)', '', df)
    df = re.sub(r'\n', '', df)
    df = re.sub(r'\[', '', df)
    df = re.sub(r'\]', '', df)
    return df

In [11]:
df_companies = pd.read_csv('tj_comp_upd.csv').rename({'index': 'doc_id'}, axis=1)
df_texts = pd.read_csv('tj_texts_upd.csv')#.rename({'0': 'text'}, axis=1)
#df_texts['preprocessed'] = df_texts.apply(preprocessing_texts, axis=1)

In [12]:
df_texts

,doc_id,text,preprocessed
0,2,Казачьим обществом «ИРБИС» в Петербурге была ...,Казачьим обществом «ИРБИС» в Петербурге была ...
1,3,Последняя версия мессенджера Whats App на And...,Последняя версия мессенджера Whats App на And...
2,4,Подросток Марсель Фернандес Фильо из Бразилии...,Подросток Марсель Фернандес Фильо из Бразилии...
3,5,Чо как дела ребят,Чо как дела ребят
4,6,Лихачев няшка. Ваши мнения?,Лихачев няшка. Ваши мнения?
...,...,...,...
298689,449996,\#tg\_arkhlikbez,\#tg\_arkhlikbez
298690,449997,Инстателочка на связи! Раз инста в ахуе загруж...,Инстателочка на связи! Раз инста в ахуе загруж...
298691,449998,NaN,NaN
298692,449999,,


In [13]:
df_companies

,doc_id,companies
0,4,samsung
1,15,apple
2,18,apple
3,20,apple
4,31,google
...,...,...
55844,449933,dns;twitter;facebook;instagram
55845,449941,dns;twitter;facebook;instagram
55846,449958,instagram;facebook
55847,449990,dns;twitter;facebook;instagram


In [8]:
df_companies['companies'].str.split(';')

0                                  [samsung]
1                                    [apple]
2                                    [apple]
3                                    [apple]
4                                   [google]
                        ...                 
55844    [dns, twitter, facebook, instagram]
55845    [dns, twitter, facebook, instagram]
55846                  [instagram, facebook]
55847    [dns, twitter, facebook, instagram]
55848                  [instagram, facebook]
Name: companies, Length: 55849, dtype: object

In [14]:
df_texts_comp = df_texts.merge(df_companies, how='left', on='doc_id')
df_texts_comp['companies'] = df_texts_comp['companies'].str.split(';')

In [67]:
df_texts_comp_preprocessed = df_texts_comp.explode('companies')#.groupby('companies').agg({'preprocessed': lambda x: ' '.join(x)}).reset_index()

In [68]:
df_texts_comp_preprocessed

,doc_id,text,preprocessed,companies
0,2,Казачьим обществом «ИРБИС» в Петербурге была ...,Казачьим обществом «ИРБИС» в Петербурге была ...,NaN
1,3,Последняя версия мессенджера Whats App на And...,Последняя версия мессенджера Whats App на And...,NaN
2,4,Подросток Марсель Фернандес Фильо из Бразилии...,Подросток Марсель Фернандес Фильо из Бразилии...,samsung
3,5,Чо как дела ребят,Чо как дела ребят,NaN
4,6,Лихачев няшка. Ваши мнения?,Лихачев няшка. Ваши мнения?,NaN
...,...,...,...,...
298689,449996,\#tg\_arkhlikbez,\#tg\_arkhlikbez,NaN
298690,449997,Инстателочка на связи! Раз инста в ахуе загруж...,Инстателочка на связи! Раз инста в ахуе загруж...,NaN
298691,449998,NaN,NaN,NaN
298692,449999,,,NaN


In [17]:
import networkx as nx
G = nx.read_gml("G_tj_upd.gml")

In [20]:
companies_vc = set(list(G.nodes))

In [69]:
df_texts_comp_preprocessed = df_texts_comp_preprocessed[df_texts_comp_preprocessed['companies'].isin(companies_vc)]#['preprocessed']

In [70]:
df_texts_comp_preprocessed

,doc_id,text,preprocessed,companies
2,4,Подросток Марсель Фернандес Фильо из Бразилии...,Подросток Марсель Фернандес Фильо из Бразилии...,samsung
10,15,Популярный ресурс про Apple и тренды перезапу...,Популярный ресурс про Apple и тренды перезапу...,apple
13,18,С сегодняшнего дня разработчики могут загрузи...,С сегодняшнего дня разработчики могут загрузи...,apple
15,20,Клуб читателей популярного ресурса про Apple ...,Клуб читателей популярного ресурса про Apple ...,apple
23,31,Google создал алгоритм для описания изображен...,Google создал алгоритм для описания изображен...,google
...,...,...,...,...
298684,449990,"Какие сервисы затронуло, предварительные причи...","Какие сервисы затронуло, предварительные причи...",twitter
298684,449990,"Какие сервисы затронуло, предварительные причи...","Какие сервисы затронуло, предварительные причи...",facebook
298684,449990,"Какие сервисы затронуло, предварительные причи...","Какие сервисы затронуло, предварительные причи...",instagram
298685,449991,Наконец-то можно запостить отложенную фотограф...,Наконец-то можно запостить отложенную фотограф...,instagram


In [26]:
df_texts_comp_preprocessed_ = df_texts_comp_preprocessed[['doc_id', 'preprocessed']].drop_duplicates(subset=['preprocessed'])

In [64]:
df_texts_comp_preprocessed[['doc_id', 'preprocessed_x']].drop_duplicates(subset=['doc_id'])

,doc_id,preprocessed_x
0,4,Подросток Марсель Фернандес Фильо из Бразилии...
1,15,Популярный ресурс про Apple и тренды перезапу...
2,18,С сегодняшнего дня разработчики могут загрузи...
3,20,Клуб читателей популярного ресурса про Apple ...
4,31,Google создал алгоритм для описания изображен...
...,...,...
94474,449933,"Какие сервисы затронуло, предварительные причи..."
94478,449941,"Какие сервисы затронуло, предварительные причи..."
94482,449958,Уже как пять с половиной часов соцсети Инстаг...
94484,449990,"Какие сервисы затронуло, предварительные причи..."


In [28]:
texts = list(df_texts_comp_preprocessed_['preprocessed'])

In [29]:
count_vect = joblib.load('./models/countVect.pkl')
dataset = count_vect.transform(texts)

lda = joblib.load('./models/lda.pkl')

In [30]:
class TopicModeler(object):
    '''
    Inteface object for CountVectorizer + LDA simple
    usage.
    '''
    def __init__(self, count_vect, lda):
        '''
        Args:
             count_vect - CountVectorizer object from sklearn.
             lda - LDA object from sklearn.
        '''
        self.lda = lda
        self.count_vect = count_vect
        self.count_vect.input = 'content'
        
    def __call__(self, text):
        '''
        Gives topics distribution for a given text
        Args:
             text - raw text via python string.
        returns: numpy array - topics distribution for a given text.
        '''
        vectorized = self.count_vect.transform([text])
        lda_topics = self.lda.transform(vectorized)
        return lda_topics
    def get_keywords(self, text, n_topics=3, n_keywords=5):
        '''
        For a given text gives n top keywords for each of m top texts topics.
        Args:
             text - raw text via python string.
             n_topics - int how many top topics to use.
             n_keywords - how many top words of each topic to return.
        returns:
                list - of m*n keywords for a given text.
        '''
        lda_topics = self(text)
        lda_topics = np.squeeze(lda_topics, axis=0)
        n_topics_indices = lda_topics.argsort()[-n_topics:][::-1]
        
        top_topics_words_dists = []
        for i in n_topics_indices:
            top_topics_words_dists.append(self.lda.components_[i])
        
        keywords = np.zeros(shape=(n_keywords*n_topics, self.lda.components_.shape[1]))
        for i,topic in enumerate(top_topics_words_dists):
            n_keywords_indices = topic.argsort()[-n_keywords:][::-1]
            for k,j in enumerate(n_keywords_indices):
                keywords[i * n_keywords + k, j] = 1
        keywords = self.count_vect.inverse_transform(keywords)
        keywords = [keyword[0] for keyword in keywords]
        return keywords, lda_topics  

In [31]:
tm = TopicModeler(count_vect, lda)

In [102]:
len(set(kw[100]))#.intersection(set(kw[1])))

406

In [33]:
kw = []
top = []
for text in tqdm(texts):
    kw_, top_ = tm.get_keywords(text, n_topics=5, n_keywords=5)
    kw.append(kw_)
    top.append(top_)

100%|███████████████████████████████████| 46187/46187 [1:50:50<00:00,  6.94it/s]


In [35]:
n_keywords = 11
keywords = np.zeros(shape=(n_keywords*60, lda.components_.shape[1]))

In [36]:
for i,topic in enumerate(lda.components_):
    n_keywords_indices = topic.argsort()[-n_keywords:][::-1]
    for k,j in enumerate(n_keywords_indices):
        keywords[i * n_keywords + k, j] = 1
keywords = count_vect.inverse_transform(keywords)
keywords = [keyword[0] for keyword in keywords]

In [37]:
keywords = np.array(keywords).reshape(60, n_keywords)

In [125]:
with open('tj_keywords_upd.npy', 'wb') as f:
    np.save(f, keywords)

In [43]:
top_max = []
for i in range(len(top)):
    top_max.append(top[i].argmax())

In [54]:
top_sort = []
for i in range(len(top)):
    top_sort.append(top[i].argsort()[-3:])

In [56]:
df_texts_comp_preprocessed_['topic'] = top_max

In [58]:
df_texts_comp_preprocessed_['topic_sorted'] = top_sort

In [59]:
df_texts_comp_preprocessed_

,doc_id,preprocessed,topic,topic_sorted
2,4,Подросток Марсель Фернандес Фильо из Бразилии...,54,"[27, 24, 54]"
10,15,Популярный ресурс про Apple и тренды перезапу...,35,"[47, 38, 35]"
13,18,С сегодняшнего дня разработчики могут загрузи...,54,"[59, 38, 54]"
15,20,Клуб читателей популярного ресурса про Apple ...,35,"[12, 38, 35]"
23,31,Google создал алгоритм для описания изображен...,48,"[53, 38, 48]"
...,...,...,...,...
298585,449866,В 2022 году на экраны выйдет сериал «Нулевой п...,22,"[14, 16, 22]"
298601,449886,"Ультиматум, который всё сложнее воспринимать в...",57,"[19, 38, 57]"
298625,449917,Система становится постепенно доступна с насту...,15,"[58, 59, 15]"
298657,449958,Уже как пять с половиной часов соцсети Инстаг...,8,"[41, 37, 8]"


In [71]:
df_texts_comp_preprocessed_top = df_texts_comp_preprocessed.merge(df_texts_comp_preprocessed_, how='left', on='preprocessed')

In [72]:
df_texts_comp_preprocessed_top

,doc_id_x,text,preprocessed,companies,doc_id_y,topic,topic_sorted
0,4,Подросток Марсель Фернандес Фильо из Бразилии...,Подросток Марсель Фернандес Фильо из Бразилии...,samsung,4,54,"[27, 24, 54]"
1,15,Популярный ресурс про Apple и тренды перезапу...,Популярный ресурс про Apple и тренды перезапу...,apple,15,35,"[47, 38, 35]"
2,18,С сегодняшнего дня разработчики могут загрузи...,С сегодняшнего дня разработчики могут загрузи...,apple,18,54,"[59, 38, 54]"
3,20,Клуб читателей популярного ресурса про Apple ...,Клуб читателей популярного ресурса про Apple ...,apple,20,35,"[12, 38, 35]"
4,31,Google создал алгоритм для описания изображен...,Google создал алгоритм для описания изображен...,google,31,48,"[53, 38, 48]"
...,...,...,...,...,...,...,...
94485,449990,"Какие сервисы затронуло, предварительные причи...","Какие сервисы затронуло, предварительные причи...",twitter,449802,41,"[15, 57, 41]"
94486,449990,"Какие сервисы затронуло, предварительные причи...","Какие сервисы затронуло, предварительные причи...",facebook,449802,41,"[15, 57, 41]"
94487,449990,"Какие сервисы затронуло, предварительные причи...","Какие сервисы затронуло, предварительные причи...",instagram,449802,41,"[15, 57, 41]"
94488,449991,Наконец-то можно запостить отложенную фотограф...,Наконец-то можно запостить отложенную фотограф...,instagram,449991,41,"[59, 54, 41]"


In [73]:
df_texts_comp_preprocessed_top.to_csv('df_texts_comp_preprocessed_top.csv', index=False)

In [77]:
comp_topic = df_texts_comp_preprocessed_top[['doc_id_x', 'companies', 'topic', 'topic_sorted']]

In [78]:
comp_topic['topic'] = comp_topic['topic'].astype(str)

/var/folders/h9/f5kwlqld0wn87p77fb7zxcpc0000gn/T/ipykernel_42874/1794505923.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_topic['topic'] = comp_topic['topic'].astype(str)


In [79]:
comp_topic

,doc_id_x,companies,topic,topic_sorted
0,4,samsung,54,"[27, 24, 54]"
1,15,apple,35,"[47, 38, 35]"
2,18,apple,54,"[59, 38, 54]"
3,20,apple,35,"[12, 38, 35]"
4,31,google,48,"[53, 38, 48]"
...,...,...,...,...
94485,449990,twitter,41,"[15, 57, 41]"
94486,449990,facebook,41,"[15, 57, 41]"
94487,449990,instagram,41,"[15, 57, 41]"
94488,449991,instagram,41,"[59, 54, 41]"


In [85]:
comp_topic['topic_sorted'] = comp_topic['topic_sorted'].apply(lambda x: ';'.join([str(item) for item in x]))

/var/folders/h9/f5kwlqld0wn87p77fb7zxcpc0000gn/T/ipykernel_42874/326385550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_topic['topic_sorted'] = comp_topic['topic_sorted'].apply(lambda x: ';'.join([str(item) for item in x]))


In [86]:
comp_topic

,doc_id_x,companies,topic,topic_sorted
0,4,samsung,54,27;24;54
1,15,apple,35,47;38;35
2,18,apple,54,59;38;54
3,20,apple,35,12;38;35
4,31,google,48,53;38;48
...,...,...,...,...
94485,449990,twitter,41,15;57;41
94486,449990,facebook,41,15;57;41
94487,449990,instagram,41,15;57;41
94488,449991,instagram,41,59;54;41


In [87]:
comp_topic = comp_topic.groupby('companies').agg({'topic': lambda x: ';'.join(x), 'topic_sorted': lambda y: ';'.join(y)}).reset_index()

In [88]:
comp_topic

,companies,topic,topic_sorted
0,2гис,45;13;57;17;30;54;37;12;20;18;8;38;56;56;56;56...,38;13;45;45;12;13;41;38;57;43;56;17;17;38;30;4...
1,a1,39;48;42;35;4;4;5;32;55;37;55;38;54;40;44;45;1...,2;35;39;5;27;48;14;44;42;31;50;35;34;9;4;20;19...
2,abacus,23;23;56;48;38;54;54,21;39;23;27;36;23;15;5;56;40;24;48;48;56;38;30...
3,abbyy,38;1;4;48;13;19;8;9;13;48;4;35;8;37;8,15;52;38;8;50;1;50;43;4;1;56;48;14;44;13;59;52...
4,abi,8;58;58;18;18;18;18;18;18,16;5;8;35;19;58;35;19;58;33;17;18;33;17;18;33;...
...,...,...,...
919,яндекс.лавка,4;13;13;43;5;13;59;59;10;12;12;52;57;37;52;52;...,41;13;4;38;12;13;38;12;13;12;16;43;16;12;5;55;...
920,яндекс.маркет,35;8;56;8;2;8;56;38;35;3;8;5;45;13;4;35;41;56;...,48;13;35;5;56;8;35;8;56;56;5;8;38;56;2;56;5;8;...
921,яндекс.услуги,35;13;13,38;13;35;56;35;13;3;57;13
922,яндекс.шеф,13;13;20;13;35;37;49,12;38;13;12;38;13;9;13;20;12;8;13;8;13;35;35;1...


In [89]:
comp_topic.to_csv('comp_topic_upd.csv', index=False)

In [102]:
from collections import defaultdict

n_comp = comp_topic.shape[0]
cnt_comp_top = list(comp_topic.topic.str.split(';'))


dict_comp_top_list = []
for i in range(n_comp):
    x = cnt_comp_top[i]
    dict_comp_top = np.zeros(60)
    for j in range(len(cnt_comp_top[i])):
        dict_comp_top[int(cnt_comp_top[i][j])] += 1
        #dict_comp_top[cnt_comp_top[i][j]] += 1
        
    dict_comp_top_list.append(dict_comp_top.copy()/dict_comp_top.sum())

In [108]:
from collections import defaultdict

n_comp = comp_topic.shape[0]
cnt_comp_top = list(comp_topic.topic_sorted.str.split(';'))


dict_comp_top_list_s = []
for i in range(n_comp):
    x = cnt_comp_top[i]
    dict_comp_top = np.zeros(60)
    for j in range(len(cnt_comp_top[i])):
        dict_comp_top[int(cnt_comp_top[i][j])] += 1
        #dict_comp_top[cnt_comp_top[i][j]] += 1
        
    dict_comp_top_list_s.append(dict_comp_top.copy()/dict_comp_top.sum())

In [109]:
data_for_pie_chart = []
for i in range(len(dict_comp_top_list)):
    s = 0
    idx = dict_comp_top_list[i].argsort()[::-1].copy()
    idx_thresh = idx[dict_comp_top_list[i][idx] > 0.1]
    data_for_pie_chart.append({})
    
    if len(idx_thresh) == 0:
        if dict_comp_top_list[i].max() == 1.0:
            idx_thresh = np.array([dict_comp_top_list[i].argmax()])
        else:
            idx_thresh = dict_comp_top_list[i].argsort()[::-1].copy()[:3]
        
    for j in idx_thresh:
        data_for_pie_chart[-1][j] = dict_comp_top_list[i][j]
        if len(data_for_pie_chart[-1].keys()) >= 3:
            break
            
    for key in data_for_pie_chart[-1].keys():
        s += data_for_pie_chart[-1][key]
    if s != 1:
        data_for_pie_chart[-1]['others'] = 1 - s

In [110]:
comp_topic['probs_sort'] = data_for_pie_chart

In [111]:
comp_topic_ = comp_topic[['companies', 'probs_','probs_sort']]

In [112]:
comp_topic_

,companies,probs_,probs_sort
0,2гис,"{56: 0.21739130434782608, 57: 0.13043478260869...","{38: 0.17391304347826086, 13: 0.13043478260869..."
1,a1,"{37: 0.12215132178669097, 'others': 0.87784867...","{37: 0.070495290185354, 59: 0.0489213005165603..."
2,abacus,"{54: 0.2857142857142857, 23: 0.285714285714285...","{23: 0.09523809523809523, 54: 0.09523809523809..."
3,abbyy,"{8: 0.2, 48: 0.13333333333333333, 4: 0.1333333...","{8: 0.1111111111111111, 'others': 0.8888888888..."
4,abi,"{18: 0.6666666666666666, 58: 0.222222222222222...","{33: 0.2222222222222222, 18: 0.222222222222222..."
...,...,...,...
919,яндекс.лавка,"{52: 0.2631578947368421, 13: 0.157894736842105...","{13: 0.15789473684210525, 35: 0.10526315789473..."
920,яндекс.маркет,"{8: 0.1875, 56: 0.125, 'others': 0.6875}","{8: 0.125, 56: 0.125, 13: 0.125, 'others': 0.625}"
921,яндекс.услуги,"{13: 0.6666666666666666, 35: 0.3333333333333333}","{13: 0.3333333333333333, 35: 0.222222222222222..."
922,яндекс.шеф,"{13: 0.42857142857142855, 35: 0.14285714285714...","{13: 0.3333333333333333, 12: 0.142857142857142..."


In [124]:
comp_topic_.to_csv('tj_topics_upd.csv', index=False)